In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import os
import polars as pl
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import pickle
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

FINAL_MODEL = None
SCALER = None
FEATURE_COLUMNS = []
HISTORY_BUFFER = None
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
SEQUENCE_LENGTH = 20

In [3]:

class LSTMPredictor(nn.Module):
   """LSTM-based model for market return prediction"""
   def __init__(self, input_dim, hidden_dim=128, num_layers=2, dropout=0.2):
        super(LSTMPredictor, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0
        )
        
        self.attention = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.Tanh(),
            nn.Linear(hidden_dim // 2, 1)
        )
        
        self.fc = nn.Sequential(
            nn.Linear(hidden_dim, 64),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(32, 1)
        )
   def forward(self, x):
        lstm_out, (hidden, cell) = self.lstm(x)
        attention_weights = torch.softmax(self.attention(lstm_out), dim=1)
        context = torch.sum(attention_weights * lstm_out, dim=1)        
        output = self.fc(context)
        return output

def convert_to_signal(predictions: np.ndarray, multiplier: float = 400.0) -> np.ndarray:
    signals = predictions * multiplier + 1
    return np.clip(signals, 0.0, 2.0)

def simple_impute(df):
    df_imp = df.copy()
    
    numeric_cols = df_imp.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        df_imp[col] = df_imp[col].fillna(method='ffill').fillna(method='bfill').fillna(0)
    
    return df_imp

def create_sequences(X, y, sequence_length):
    """Create sequences for LSTM training"""
    X_seq = []
    y_seq = []
    
    for i in range(len(X) - sequence_length):
        X_seq.append(X[i:i + sequence_length])
        y_seq.append(y[i + sequence_length])
    
    return np.array(X_seq), np.array(y_seq)

def train_model():
    global FINAL_MODEL, SCALER, FEATURE_COLUMNS, HISTORY_BUFFER, SEQUENCE_LENGTH
    
    print("Training LSTM model...")
    print(f"Using device: {DEVICE}")
    start_time = pd.Timestamp.now()
    
    path = "./kaggle"
    print(f"Loading data from {path}")
    
    training_df = pd.read_csv(f"{path}/train.csv")
    print(f"Loaded {len(training_df)} rows")
    
    print("Imputing missing values...")
    training_df = simple_impute(training_df)
    print(f"Imputation done. Remaining NaN: {training_df.isnull().sum().sum()}")
    
    if isinstance(training_df, pd.DataFrame):
        training_df = pl.from_pandas(training_df)
    
    training_df = training_df.rename({'market_forward_excess_returns': 'target'})
    
    feature_cols = [col for col in training_df.columns if col != 'date_id']
    training_df = training_df.with_columns(pl.col(feature_cols).cast(pl.Float64, strict=False))
    
    training_df = training_df.drop_nulls()
    print(f"After dropping nulls: {len(training_df)} rows")
    
    FEATURE_COLUMNS = [col for col in training_df.columns 
                       if col not in ['date_id', 'target', 'forward_returns', 'risk_free_rate']]
    
    print(f"Training with {len(FEATURE_COLUMNS)} features")
    
    X_train = training_df.select(FEATURE_COLUMNS).to_numpy()
    y_train = training_df.select('target').to_numpy().ravel()
    
    SCALER = StandardScaler()
    X_train_scaled = SCALER.fit_transform(X_train)
    
    print(f"Creating sequences with length {SEQUENCE_LENGTH}...")
    X_seq, y_seq = create_sequences(X_train_scaled, y_train, SEQUENCE_LENGTH)
    print(f"Created {len(X_seq)} sequences")
    
    X_tensor = torch.FloatTensor(X_seq).to(DEVICE)
    y_tensor = torch.FloatTensor(y_seq).reshape(-1, 1).to(DEVICE)
    
    dataset = TensorDataset(X_tensor, y_tensor)
    train_loader = DataLoader(dataset, batch_size=128, shuffle=True)
    
    input_dim = len(FEATURE_COLUMNS)
    FINAL_MODEL = LSTMPredictor(
        input_dim=input_dim,
        hidden_dim=128,
        num_layers=2,
        dropout=0.3
    ).to(DEVICE)
    
    criterion = nn.MSELoss()
    optimizer = optim.Adam(FINAL_MODEL.parameters(), lr=0.001, weight_decay=1e-5)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)
    
    num_epochs = 100
    best_loss = float('inf')
    patience = 15
    patience_counter = 0
    
    print("Starting training...")
    FINAL_MODEL.train()
    
    for epoch in range(num_epochs):
        total_loss = 0
        
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            
            predictions = FINAL_MODEL(batch_X)
            
            loss = criterion(predictions, batch_y)
            
            loss.backward()
            
            torch.nn.utils.clip_grad_norm_(FINAL_MODEL.parameters(), max_norm=1.0)
            
            optimizer.step()
            
            total_loss += loss.item()
        
        avg_loss = total_loss / len(train_loader)
        scheduler.step(avg_loss)
        
        if (epoch + 1) % 10 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.6f}")
        
        if avg_loss < best_loss:
            best_loss = avg_loss
            patience_counter = 0
        else:
            patience_counter += 1
        
        if patience_counter >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break
    
    FINAL_MODEL.eval()
    print(f"Model training complete! Best Loss: {best_loss:.6f}")
    
    buffer_size = SEQUENCE_LENGTH + 50
    HISTORY_BUFFER = training_df.select(
        ['date_id'] + FEATURE_COLUMNS + ['target', 'forward_returns', 'risk_free_rate']
    ).tail(buffer_size)
    
    elapsed = (pd.Timestamp.now() - start_time).total_seconds()
    print(f"Model training complete in {elapsed:.1f} seconds. Ready for predictions.")

def predict(test: pl.DataFrame) -> float:
    """
    Make a prediction for a single test row.
    
    Args:
        test: A polars DataFrame with one row containing the test features
        
    Returns:
        A float signal value between 0 and 2
    """
    global HISTORY_BUFFER, FINAL_MODEL, SCALER, FEATURE_COLUMNS, SEQUENCE_LENGTH
    
    try:
        feature_cols = [col for col in test.columns if col != 'date_id']
        test = test.with_columns(pl.col(feature_cols).cast(pl.Float64, strict=False))
        
        rename_mapping = {}
        if 'lagged_forward_returns' in test.columns:
            rename_mapping['lagged_forward_returns'] = 'forward_returns'
        if 'lagged_risk_free_rate' in test.columns:
            rename_mapping['lagged_risk_free_rate'] = 'risk_free_rate'
        if 'lagged_market_forward_excess_returns' in test.columns:
            rename_mapping['lagged_market_forward_excess_returns'] = 'target'
        
        if rename_mapping:
            test = test.rename(rename_mapping)
        
        if 'is_scored' in test.columns:
            test = test.drop('is_scored')
        
        for col in HISTORY_BUFFER.columns:
            if col not in test.columns:
                test = test.with_columns(pl.lit(0.0).cast(pl.Float64).alias(col))
        
        test = test.select(HISTORY_BUFFER.columns)
        
        HISTORY_BUFFER = pl.concat([HISTORY_BUFFER, test], how="vertical")
        
        max_buffer = SEQUENCE_LENGTH + 100
        if len(HISTORY_BUFFER) > max_buffer:
            HISTORY_BUFFER = HISTORY_BUFFER.tail(max_buffer)
        
        if len(HISTORY_BUFFER) < SEQUENCE_LENGTH:
            needed = SEQUENCE_LENGTH - len(HISTORY_BUFFER)
            first_row = HISTORY_BUFFER.head(1)
            padding = pl.concat([first_row] * needed, how="vertical")
            sequence_data = pl.concat([padding, HISTORY_BUFFER], how="vertical")
        else:
            sequence_data = HISTORY_BUFFER.tail(SEQUENCE_LENGTH)
        
        X_sequence = sequence_data.select(FEATURE_COLUMNS).to_numpy()
        
        X_sequence = np.nan_to_num(X_sequence, nan=0.0)
        
        X_sequence_scaled = SCALER.transform(X_sequence)
        
        X_sequence_scaled = X_sequence_scaled.reshape(1, SEQUENCE_LENGTH, -1)
        
        X_tensor = torch.FloatTensor(X_sequence_scaled).to(DEVICE)
        
        with torch.no_grad():
            raw_prediction = FINAL_MODEL(X_tensor).cpu().numpy()[0, 0]
        
        signal = convert_to_signal(np.array([raw_prediction]))[0]
        
        return float(signal)
        
    except Exception as e:
        print(f"Error in prediction: {e}")
        import traceback
        traceback.print_exc()
        return 1.0


In [4]:


print("="*50)
print("Starting LSTM model training...")
print("="*50)
train_model()
print("="*50)
print("Model ready. Starting inference server...")
print("="*50)

import kaggle_evaluation.default_inference_server

inference_server = kaggle_evaluation.default_inference_server.DefaultInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    print("Running in competition mode")
    inference_server.serve()
else:
    print("Running in local gateway mode")
    inference_server.run_local_gateway(
        ('./kaggle',)
    )

Starting LSTM model training...
Training LSTM model...
Using device: cuda
Loading data from ./kaggle
Loaded 8990 rows
Imputing missing values...
Imputation done. Remaining NaN: 0
After dropping nulls: 8990 rows
Training with 94 features
Creating sequences with length 20...
Created 8970 sequences


/tmp/ipykernel_11679/88806073.py:48: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_imp[col] = df_imp[col].fillna(method='ffill').fillna(method='bfill').fillna(0)
/home/paul/anaconda3/envs/ml_311/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Starting training...
Epoch [10/100], Loss: 0.000138
Epoch [20/100], Loss: 0.000112
Epoch [30/100], Loss: 0.000111
Epoch [40/100], Loss: 0.000112
Early stopping at epoch 47
Model training complete! Best Loss: 0.000111
Model training complete in 6.4 seconds. Ready for predictions.
Model ready. Starting inference server...
Running in local gateway mode


In [5]:
# ==================================================
# Model Performance Evaluation (Last 180 Days)
# ==================================================

# --- 1. Metric Implementation ---

def calculate_competition_score(y_true_df: pl.DataFrame, y_pred_signals: np.ndarray) -> float:
    """
    Compute the Adjusted Sharpe Ratio for the competition.
    Requires 'forward_returns' and 'risk_free_rate' in y_true_df.
    """
    solution = y_true_df.to_pandas()
    solution['position'] = y_pred_signals

    # --- Strategy Returns Calculation ---
    solution['strategy_returns'] = (
        solution['risk_free_rate'] * (1 - solution['position']) +
        solution['position'] * solution['forward_returns']
    )
    strategy_excess_returns = solution['strategy_returns'] - solution['risk_free_rate']
    strategy_geo_mean = (1 + strategy_excess_returns).prod() ** (1 / len(solution)) - 1
    strategy_std = solution['strategy_returns'].std()
    
    if strategy_std == 0: return 0.0
    
    trading_days_per_yr = 252
    sharpe = strategy_geo_mean / strategy_std * np.sqrt(trading_days_per_yr)
    
    # --- Market Comparison and Volatility Penalty ---
    market_std = solution['forward_returns'].std()
    market_volatility = market_std * np.sqrt(trading_days_per_yr) * 100
    strategy_volatility = strategy_std * np.sqrt(trading_days_per_yr) * 100
    
    excess_vol = max(0, strategy_volatility / market_volatility - 1.2) if market_volatility > 0 else 0
    vol_penalty = 1 + excess_vol
    
    # --- Return Gap Penalty ---
    market_excess_returns = solution['forward_returns'] - solution['risk_free_rate']
    market_geo_mean = (1 + market_excess_returns).prod() ** (1 / len(solution)) - 1
    
    return_gap = max(0, (market_geo_mean - strategy_geo_mean) * 100 * trading_days_per_yr)
    return_penalty = 1 + (return_gap**2) / 100
    
    adjusted_sharpe = sharpe / (vol_penalty * return_penalty)
    return adjusted_sharpe


# --- 2. Data Preparation for Validation ---

# Get the full training data (which includes the buffer and more)
# We assume the 'training_df' (before the tail() for HISTORY_BUFFER)
# represents all pre-processed data. We will re-read it for clean slicing.

print("\n" + "="*50)
print("Evaluating LSTM on the last 180 days (Validation Set)...")
print("="*50)

# Re-load data, since training_df was filtered/converted locally in train_model()
path = "./kaggle"
full_df = pd.read_csv(f"{path}/train.csv").rename({'market_forward_excess_returns': 'target'}, axis=1)

# Apply the same imputation used in training
def simple_impute_pd(df):
    df_imp = df.copy()
    numeric_cols = df_imp.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        # Use ffill/bfill replacement for the now deprecated fillna(method)
        df_imp[col] = df_imp[col].ffill().bfill().fillna(0)
    return df_imp

full_df = simple_impute_pd(full_df)

# Convert to Polars for slicing and metric
full_pl_df = pl.from_pandas(full_df)

# Prepare the validation set (last 180 rows)
VALIDATION_SIZE = 180
val_pl_df = full_pl_df.tail(VALIDATION_SIZE)

# Get features for scaling
X_val = val_pl_df.select(FEATURE_COLUMNS).to_numpy()
y_val_info = val_pl_df.select(["target", "forward_returns", "risk_free_rate"])

# Scale the features
X_val_scaled = SCALER.transform(X_val)

# --- 3. Sequence Creation for Validation ---

# NOTE: For proper time-series validation, the validation sequence
# must include SEQUENCE_LENGTH-1 days of lookback *from the training data*.
# The create_sequences function handles this automatically if we use the right slice.
# However, for *inference* on the full validation block, we create sequences
# differently to get one prediction per day.

def create_inference_sequences(X, sequence_length):
    X_seq = []
    # Loop from the first day where a full sequence can be formed
    for i in range(len(X) - sequence_length + 1):
        X_seq.append(X[i:i + sequence_length])
    return np.array(X_seq)

# We need data for the full validation period (180 days) plus the sequence length
# from the end of the training data.
needed_history = SEQUENCE_LENGTH - 1
start_index = len(full_pl_df) - VALIDATION_SIZE - needed_history

# Slice the data to include the lookback for the first validation day
full_slice = full_pl_df.slice(start_index, VALIDATION_SIZE + needed_history)
X_full_slice = full_slice.select(FEATURE_COLUMNS).to_numpy()

# Scale the full slice
X_full_slice_scaled = SCALER.transform(X_full_slice)

# Create sequences for the 180 days of validation
X_val_seq = create_inference_sequences(X_full_slice_scaled, SEQUENCE_LENGTH)

# --- 4. Prediction Generation ---
FINAL_MODEL.eval()
X_val_tensor = torch.FloatTensor(X_val_seq).to(DEVICE)

with torch.no_grad():
    raw_predictions = FINAL_MODEL(X_val_tensor).cpu().numpy().flatten()

# Convert to signal
val_signals = convert_to_signal(raw_predictions)

# --- 5. Score Calculation ---
val_score = calculate_competition_score(y_val_info, val_signals)

print(f"Validation Score (Adjusted Sharpe) on last {VALIDATION_SIZE} days: {val_score:.4f}")
print("="*50)

# Note: The output of this block will appear after the training output
# in the final execution of the complete notebook.


Evaluating LSTM on the last 180 days (Validation Set)...
Validation Score (Adjusted Sharpe) on last 180 days: 0.4583


In [7]:
# ==================================================
# LSTM Time Series Cross-Validation (CV)
# ==================================================

import polars as pl
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit

# --- 1. Utility/Metric Functions (Re-defined for CV scope) ---

def convert_to_signal(predictions: np.ndarray, multiplier: float = 400.0) -> np.ndarray:
    """Converts model predictions (excess returns) to the trading signal (0 to 2)."""
    signals = predictions * multiplier + 1
    return np.clip(signals, 0.0, 2.0)

def calculate_competition_score(y_true_df: pl.DataFrame, y_pred_signals: np.ndarray) -> float:
    """Computes the Adjusted Sharpe Ratio (ASR) metric."""
    solution = y_true_df.to_pandas()
    solution['position'] = y_pred_signals

    solution['strategy_returns'] = (
        solution['risk_free_rate'] * (1 - solution['position']) +
        solution['position'] * solution['forward_returns']
    )
    strategy_excess_returns = solution['strategy_returns'] - solution['risk_free_rate']
    strategy_geo_mean = (1 + strategy_excess_returns).prod() ** (1 / len(solution)) - 1
    strategy_std = solution['strategy_returns'].std()
    
    if strategy_std == 0: return 0.0
    
    trading_days_per_yr = 252
    sharpe = strategy_geo_mean / strategy_std * np.sqrt(trading_days_per_yr)
    
    market_std = solution['forward_returns'].std()
    market_volatility = market_std * np.sqrt(trading_days_per_yr) * 100
    strategy_volatility = strategy_std * np.sqrt(trading_days_per_yr) * 100
    
    excess_vol = max(0, strategy_volatility / market_volatility - 1.2) if market_volatility > 0 else 0
    vol_penalty = 1 + excess_vol
    
    market_excess_returns = solution['forward_returns'] - solution['risk_free_rate']
    market_geo_mean = (1 + market_excess_returns).prod() ** (1 / len(solution)) - 1
    
    return_gap = max(0, (market_geo_mean - strategy_geo_mean) * 100 * trading_days_per_yr)
    return_penalty = 1 + (return_gap**2) / 100
    
    adjusted_sharpe = sharpe / (vol_penalty * return_penalty)
    return float(min(adjusted_sharpe, 1_000_000))

def create_inference_sequences(X, sequence_length):
    """Creates sequences for batch prediction (one output per sequence)."""
    X_seq = []
    # Loop from the first day where a full sequence can be formed
    for i in range(len(X) - sequence_length + 1):
        X_seq.append(X[i:i + sequence_length])
    return np.array(X_seq)

# --- 2. Training Function Wrapper ---

def train_lstm_fold(X_train_fold, y_train_fold, input_dim, sequence_length, device):
    """Trains the LSTM model for one cross-validation fold."""
    
    # Sequence creation
    def create_sequences_fold(X, y, seq_len):
        X_seq, y_seq = [], []
        for i in range(len(X) - seq_len):
            X_seq.append(X[i:i + seq_len])
            y_seq.append(y[i + seq_len])
        return np.array(X_seq), np.array(y_seq)
        
    X_seq, y_seq = create_sequences_fold(X_train_fold, y_train_fold, sequence_length)
    
    X_tensor = torch.FloatTensor(X_seq).to(device)
    y_tensor = torch.FloatTensor(y_seq).reshape(-1, 1).to(device)
    
    dataset = TensorDataset(X_tensor, y_tensor)
    train_loader = DataLoader(dataset, batch_size=128, shuffle=False)
    
    # Model instantiation
    # (Note: Assumes LSTMPredictor class is defined in the notebook's global scope)
    model = LSTMPredictor(
        input_dim=input_dim, hidden_dim=256, num_layers=3, dropout=0.3
    ).to(device)
    
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)    
    num_epochs = 100
    best_loss = float('inf')
    patience = 15
    patience_counter = 0
    
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            predictions = model(batch_X)
            loss = criterion(predictions, batch_y)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            total_loss += loss.item()
        
        avg_loss = total_loss / len(train_loader)
        scheduler.step(avg_loss)
        
        if avg_loss < best_loss:
            best_loss = avg_loss
            patience_counter = 0
        else:
            patience_counter += 1
        
        if patience_counter >= patience:
            # print(f"Early stopping at epoch {epoch+1}")
            break
            
    model.eval()
    return model

# --- 3. CV Execution ---

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {DEVICE}")
SEQUENCE_LENGTH = 30
PATH = "./kaggle"

# Re-load data and apply imputation/prep as done in train_model
print("Loading and preparing full training data for CV...")
full_df = pd.read_csv(f"{PATH}/train.csv").rename({'market_forward_excess_returns': 'target'}, axis=1)


def simple_impute_pd(df):
    df_imp = df.copy()
    numeric_cols = df_imp.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        df_imp[col] = df_imp[col].ffill().bfill().fillna(0)
    return df_imp
full_df = simple_impute_pd(full_df).drop(columns=['date_id'])
full_pl_df = pl.from_pandas(full_df).drop_nulls()

full_pl_df = full_pl_df.slice(3000) # Optional: Skip initial rows if needed

FEATURE_COLUMNS = [col for col in full_pl_df.columns 
                   if col not in ['target', 'forward_returns', 'risk_free_rate']]

# Prepare data for CV
X_all = full_pl_df.select(FEATURE_COLUMNS).to_numpy()
y_all = full_pl_df.select('target').to_numpy().ravel()
scorer_info_df_all = full_pl_df.select(["forward_returns", "risk_free_rate", "target"])

# Scaling: Fit on the entire data, as the original scaler was fit on the whole train.csv
SCALER_CV = StandardScaler()
X_all_scaled = SCALER_CV.fit_transform(X_all)

print("\n" + "="*50)
nsplits = 20
print(f"Starting {nsplits}-Fold Time Series Cross-Validation (LSTM)...")
print("="*50)

tscv = TimeSeriesSplit(n_splits=nsplits)
cv_scores = []

for i, (train_index, test_index) in enumerate(tscv.split(X_all_scaled)):
    
    # --- Split Data ---
    # We need a lookback history for the test set, taken from the end of the train set
    needed_history = SEQUENCE_LENGTH - 1
    
    # X_train is the standard split for direct model training
    X_train_fold = X_all_scaled[train_index]
    y_train_fold = y_all[train_index]
    
    # X_test_full includes the lookback period from the train set
    X_test_start_index = test_index[0] - needed_history
    X_test_end_index = test_index[-1] + 1
    X_test_full = X_all_scaled[X_test_start_index:X_test_end_index]
    
    # y_test_info is just the evaluation window
    y_test_info = scorer_info_df_all[test_index]
    
    # --- Train Model ---
    model = train_lstm_fold(
        X_train_fold, y_train_fold, len(FEATURE_COLUMNS), SEQUENCE_LENGTH, DEVICE
    )
    
    # --- Predict and Score ---
    # Create sequences for the test set
    X_test_seq = create_inference_sequences(X_test_full, SEQUENCE_LENGTH)
    
    X_test_tensor = torch.FloatTensor(X_test_seq).to(DEVICE)
    
    with torch.no_grad():
        raw_predictions = model(X_test_tensor).cpu().numpy().flatten()
    
    signals = convert_to_signal(raw_predictions)
    score = calculate_competition_score(y_test_info, signals)
    
    cv_scores.append(score)
    print(f"  Fold {i+1}/{nsplits} Score: {score:.4f} (Test size: {len(test_index)} days)")

print(f"\nMean CV Score: {np.mean(cv_scores):.4f} (+/- {np.std(cv_scores):.4f})")
print("="*50)

Using device: cuda
Loading and preparing full training data for CV...

Starting 20-Fold Time Series Cross-Validation (LSTM)...
  Fold 1/20 Score: 0.8628 (Test size: 285 days)
  Fold 2/20 Score: -0.4024 (Test size: 285 days)
  Fold 3/20 Score: 0.4892 (Test size: 285 days)
  Fold 4/20 Score: 1.0573 (Test size: 285 days)
  Fold 5/20 Score: -0.8845 (Test size: 285 days)
  Fold 6/20 Score: -0.1118 (Test size: 285 days)
  Fold 7/20 Score: 1.0062 (Test size: 285 days)
  Fold 8/20 Score: 0.5139 (Test size: 285 days)
  Fold 9/20 Score: 1.2330 (Test size: 285 days)
  Fold 10/20 Score: 1.5954 (Test size: 285 days)
  Fold 11/20 Score: 0.9013 (Test size: 285 days)
  Fold 12/20 Score: 0.3992 (Test size: 285 days)
  Fold 13/20 Score: 1.8171 (Test size: 285 days)
  Fold 14/20 Score: 0.2820 (Test size: 285 days)
  Fold 15/20 Score: 1.2368 (Test size: 285 days)
  Fold 16/20 Score: 0.9574 (Test size: 285 days)
  Fold 17/20 Score: 0.7790 (Test size: 285 days)
  Fold 18/20 Score: -0.2302 (Test size: 285 da